##  Predict your coarse-grained genre 

4 genres : 


-  Pop/Rock : Rock, Pop, Electronic (musique populaire et électro).
- Urbain/Black music : Hip-Hop, Soul-RnB, Blues (genres liés aux musiques afro‑américaines modernes).
- Acoustique / Traditionnel : Folk, Country, Old-Time / Historic, International, Easy Listening (musiques plus “roots”, acoustiques ou de tradition locale).
- Spécialisé / Instrumental : Instrumental, Classical, Jazz, Spoken, Experimental (musiques plutôt instrumentales, “art” ou parlées).​

In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier




In [6]:

df = pd.read_csv("df_phase1.csv")
df_classif = pd.read_csv("df_classif_phase1.csv")
df_audio = pd.read_csv("df_audio_phase1.csv")


In [7]:
# 1. Définir une taxonomie "coarse" à 4 genres

coarse_map = {
    "Rock": "Pop/Rock",
    "Pop": "Pop/Rock",
    "Electronic": "Pop/Rock",

    "Hip-Hop": "Urbain",
    "Soul-RnB": "Urbain",
    "Blues": "Urbain",

    "Folk": "Acoustique",
    "Country": "Acoustique",
    "Old-Time / Historic": "Acoustique",
    "International": "Acoustique",
    "Easy Listening": "Acoustique",

    "Instrumental": "Spécialisé",
    "Classical": "Spécialisé",
    "Jazz": "Spécialisé",
    "Spoken": "Spécialisé",
    "Experimental": "Spécialisé",
}

df_classif["genre_coarse"] = df_classif["genre_top"].map(coarse_map)
print(df_classif["genre_coarse"].value_counts(dropna=False))


genre_coarse
Pop/Rock      23089
Spécialisé    13691
Acoustique     4309
Urbain         3544
Name: count, dtype: int64


In [8]:
# 2. Encoder la cible coarse
le_coarse = LabelEncoder()
df_classif["genre_coarse_encoded"] = le_coarse.fit_transform(df_classif["genre_coarse"])

# 2b. Définir un set de features de base
feature_cols = [
    "log_duration", "log_listens", "log_favorites",
    "favorites_per_listen",
    "tempo", "energy", "danceability"
]
feature_cols = [c for c in feature_cols if c in df_classif.columns]

X = df_classif[feature_cols].fillna(0)
y = df_classif["genre_coarse_encoded"]

print("Features utilisées :", feature_cols)
print("X shape :", X.shape, " | y shape :", y.shape)

Features utilisées : ['log_duration', 'log_listens', 'log_favorites', 'favorites_per_listen', 'tempo', 'energy', 'danceability']
X shape : (44633, 7)  | y shape : (44633,)


on a 44 633 morceaux, 7 features et 4 classes coarse

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train X :", X_train.shape, " | Train y :", y_train.shape)
print("Test  X :", X_test.shape,  " | Test y :", y_test.shape)


Train X : (35706, 7)  | Train y : (35706,)
Test  X : (8927, 7)  | Test y : (8927,)


In [12]:
models = {
    "LogReg": LogisticRegression(max_iter=1000, n_jobs=-1),
    "kNN": KNeighborsClassifier(n_neighbors=15),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    ),
}

for name, clf in models.items():
    print(f"\n=== {name} ===")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred, digits=3))



=== LogReg ===
              precision    recall  f1-score   support

           0      0.409     0.055     0.096       862
           1      0.565     0.907     0.696      4618
           2      0.659     0.335     0.444      2738
           3      0.167     0.001     0.003       709

    accuracy                          0.577      8927
   macro avg      0.450     0.324     0.310      8927
weighted avg      0.547     0.577     0.506      8927


=== kNN ===
              precision    recall  f1-score   support

           0      0.244     0.036     0.063       862
           1      0.581     0.819     0.680      4618
           2      0.573     0.467     0.515      2738
           3      0.239     0.023     0.041       709

    accuracy                          0.572      8927
   macro avg      0.409     0.336     0.325      8927
weighted avg      0.519     0.572     0.519      8927


=== RandomForest ===
              precision    recall  f1-score   support

           0      0.243 

- Les trois modèles arrivent à un accuracy autour de 0.55–0.58, donc mieux qu’au hasard mais encore loin d’être parfait, et tous ont tendance à favoriser la classe majoritaire (classe 1 : Pop/Rock) avec un recall très élevé pour celle‑ci.

- Les classes minoritaires sont peu reconnues : les recalls sont proches de 0, ce qui montre qu’il y a encore un gros déséquilibre entre les 4 genres coarse et qu’il faudra probablement tester un modèle plus puissant (SVM, boosting) pour mieux capturer ces catégories.

​

In [14]:
models_more = {
    "SVM_RBF": SVC(kernel="rbf", C=3, gamma="scale"),
    "GradBoost": GradientBoostingClassifier(random_state=42)
}

for name, clf in models_more.items():
    print(f"\n=== {name} ===")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred, digits=3))



=== SVM_RBF ===


c:\Users\Nada\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Nada\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Nada\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

              precision    recall  f1-score   support

           0      0.000     0.000     0.000       862
           1      0.518     1.000     0.683      4618
           2      0.913     0.008     0.015      2738
           3      0.000     0.000     0.000       709

    accuracy                          0.519      8927
   macro avg      0.358     0.252     0.174      8927
weighted avg      0.548     0.519     0.358      8927


=== GradBoost ===
              precision    recall  f1-score   support

           0      0.553     0.090     0.156       862
           1      0.595     0.849     0.700      4618
           2      0.623     0.487     0.547      2738
           3      0.707     0.058     0.107       709

    accuracy                          0.602      8927
   macro avg      0.620     0.371     0.377      8927
weighted avg      0.609     0.602     0.553      8927



- Le SVM se comporte très mal : il prédit presque tout en classe 1 (Pop/Rock), avec un recall de 1.0 pour cette classe mais quasi 0 pour les autres, ce qui donne une accuracy moyenne (0.52) mais une macro‑F1 catastrophique, donc on écarte le modèle.

​- Le Gradient Boosting est meilleur : l’accuracy monte autour de 0.60, les classes 1 et 2 sont plutôt bien reconnues, mais les classes minoritaires (0 et 3) restent difficiles avec des recalls faibles, ce qui confirme que le regroupement en 4 genres aide un peu, mais que le déséquilibre de classes reste un problème important

### Conclusion!

- La taxonomie genre_coarse regroupe les 16 genres initiaux en 4 familles (Pop/Rock, Urbain, Acoustique, Spécialisé), ce qui réduit la complexité du problème tout en gardant une interprétation musicale claire.

- Sur ces 4 classes, les modèles simples (LogReg, kNN, RandomForest) atteignent une accuracy autour de 0.55–0.58, avec un bon recall pour la classe majoritaire mais des performances limitées sur les genres minoritaires.

​- Le Gradient Boosting obtient les meilleurs résultats avec une accuracy ≈ 0.60, montrant que la réduction du nombre de classes améliore légèrement la généralisation par rapport à la Task 1, même si le déséquilibre entre familles de genres reste un frein important. (à vérifier avec la TASK1)
​